# partlist-io2md

A tool to create a Markdown part list from a BOM (Bill Of Materials) extracted
from a project designed with [Stud.IO CAD](https://www.bricklink.com/v3/studio/download.page).
For each submodel in the project a dedicated list is created inside the markdown file.

> MIT License
> 
> Copyright &copy; 2022 by Alessandro Varesi
> 
> Permission is hereby granted, free of charge, to any person obtaining a copy
> of this software and associated documentation files (the "Software"), to deal
> in the Software without restriction, including without limitation the rights
> to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
> copies of the Software, and to permit persons to whom the Software is
> furnished to do so, subject to the following conditions:
>
> The above copyright notice and this permission notice shall be included in all
> copies or substantial portions of the Software.
>
> THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
> IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
> FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
> AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
> LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
> OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
> SOFTWARE.
>

In [3]:
%%capture

import sys
# Install pip packages in current Jupyter kernel
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install urllib
!{sys.executable} -m pip install beautifulsoup4

## Choose the IO file name

In next code cell set the CSV filename (with path if needed)

In [4]:
iofile = "../../sets/10298/10298-bags.io"
colorsfile = "StudioColorDefinition.txt"


Now `Run all` cells of this notebook

In [5]:
import zipfile
import pandas as pd
from IPython.display import display, Markdown

with zipfile.ZipFile(iofile) as z:
    # print (z.infolist())

    with z.open('.info', pwd = b'soho0909') as f:
        print ('==== .info ====')
        print (f.readlines())

    model_file = z.open('model.ldr',  pwd = b'soho0909')
    print ('==== model.ldr ====')
    for l in model_file.readlines():
        print (l.strip().decode('utf-8-sig'))

    model_file.seek(0)

==== .info ====
[b'\xef\xbb\xbf{"version":"2.22.2_1\\r","total_parts":888}']
==== model.ldr ====
0 FILE 10281-bags
0 Untitled Model
0 Name:  10281-bags
0 Author:
0 CustomBrick
1 16 90.000000 0.000000 -380.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 1
1 16 480.000000 0.000000 -320.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 2
1 16 770.000000 0.000000 -90.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 3
1 16 1020.000000 0.000000 -170.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 4
1 16 110.000000 0.000000 -1100.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 5
1 16 740.000000 4.000000 -1120.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 6
1 16 1120.000000 4.000000 -1000.000000 1.000000 0.000000 0.000000 0.000000 1.00

In [6]:
# Get color information from StudioColorDefinition.txt 
# ... this file is from Stud.io application
import math


colors = pd.read_csv(colorsfile, sep='\t')
# colors


def getColorInfo(color: int):
    """Get 'BL Color Code' and 'BL Color Name'

    ### Parameter
    color : int
        the LDraw color code

    ### Returns
    bl_color : dictionary
        BL Color Code : int
            the Bricklink color code (integer)
        BL Color Name : str 
            the Bricklinr color name (str)
    
    """

    # (colors['BL Color Code'] == colors['BL Color Code']) is atrick to check is not a NaN
    r = colors.loc[(colors['LDraw Color Code'] == color) & (colors['BL Color Code'] == colors['BL Color Code'])]

    return {
        'BL Color Code': int(r.iloc[0]['BL Color Code']),
        'BL Color Name': r.iloc[0]['BL Color Name']
    }

# colors = colors[['LDraw Color Code','BL Color Code','BL Color Name']]

print(getColorInfo(84))
print(getColorInfo(28))

{'BL Color Code': 150, 'BL Color Name': 'Medium Nougat'}
{'BL Color Code': 69, 'BL Color Name': 'Dark Tan'}


In [ ]:
# Get various information from Bricklink site by getting info direct from html pages
# Not using Official API that need a AccessKey given only to registered sellers users.
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import json
import re
import os

infocachefile = ".cache/parts-info.json"
altpartsfile = "parts-alternative.json" 

# create the folder 


try:
    infocache = json.load(open(infocachefile))
except IOError:
    infocache = {}

try:
    altparts = json.load(open(altpartsfile))
except IOError:
    altparts = {}

def getPartInfo(part : str, blcolor: int):
    if part in altparts:
        orig_part = altparts[part]
    else:
        orig_part = part

    if orig_part in infocache:
        ret = infocache[orig_part].copy()
        ret['image'] = f"https://img.bricklink.com/ItemImage/PN/{blcolor}/{orig_part}.png"
        ret['partname'] = part
        return ret

    url_catalog = f'https://www.bricklink.com/v2/catalog/catalogitem.page?P={orig_part}'
    url_search = f'https://www.bricklink.com/catalogList.asp?q={orig_part}&catType=P&catID='
    req = Request(url_catalog, headers={'User-Agent': 'Mozilla/5.0'})
    with urlopen(req, timeout=10) as res:
        if re.search("notFound.asp", res.url):
            req = Request(url_search, headers={'User-Agent': 'Mozilla/5.0'})
            with urlopen(req, timeout=10) as res:
                html = res.read()
                url = res.url
        else:
            html = res.read()
            url = res.url

    soup = BeautifulSoup(html, 'html.parser')
    m = re.search(r"Part (.*) \|", soup.title.text)
    if m :
        orig_part = m[1].strip()
    else:
        orig_part = None

    description = soup.find('h1', {'id': 'item-name-title'})
    description = description.string if description else ''
    weight = soup.find('span', {'id': 'item-weight-info'})
    weight = weight.string if weight else ''
    dimension = soup.find('span', {'id': 'dimSec'})
    dimension = dimension.string if dimension else ''
    image = f'https://img.bricklink.com/ItemImage/PN/{blcolor}/{orig_part}.png'
    image = image if description else ''

    ret = {
        'original_partname' : orig_part,
        'description': description, 
        'weight': weight,
        'dimension': dimension,
        'url' : url
    }

    if orig_part:
        # Check if a new alternative part must be stored
        if orig_part != part:
            altparts[part] = orig_part
            # create the folder if inexistent
            # os.makedirs(os.path.dirname(altpartsfile), exist_ok=True)
            with open(altpartsfile, 'w') as outfile:
                json.dump(altparts, outfile, sort_keys=True, indent=4)
        infocache[orig_part] = ret.copy()
        os.makedirs(os.path.dirname(infocachefile), exist_ok=True)
        with open(infocachefile, 'w') as outfile:
            json.dump(infocache, outfile, sort_keys=True, indent=4)
    
    ret['image'] = f"https://img.bricklink.com/ItemImage/PN/{blcolor}/{ret['original_partname']}.png"
    #ret['image'] = f"https://img.bricklink.com/P/{blcolor}/{ret['original_partname']}.jpg"
    
    ret['partname'] = part

    return ret

print(getPartInfo('96874', 4))

InvalidURL: URL can't contain control characters. '/v2/catalog/catalogitem.page?P=96874 |' (found at least ' ')

In [10]:
# Scan model file and fill the BOM dictionary structure
# part = { 
# }

submodels = {}
parts = {}

class Model:
    def __init__(self, name: str):
        self.name = name
        self.BOM = {"parts": {}, "submodels": {}}
        # self.submodels = {}

    def addSubmodel(self, sm_name: str):
        sm_id = sm_name.lower()
        if sm_id not in submodels:
            print (f"{self.name}.addSubmodel({sm_name})")
            submodels[sm_id] = Model(sm_name)
        self.BOM["submodels"][sm_id] = submodels[sm_id].BOM

    def addPart(self, part: str, color: int):
        part_id = f'{color}_{part}'
        #print (f"{self.name}.addPart({part_id}) ",end='') #DEBUG
        if part_id not in parts:
        #    print (f"LegoPart({part}, {color}) ",end='')
            parts[part_id] = LegoPart(part, color)
        if part_id in self.BOM["parts"]:
            self.BOM["parts"][part_id]['qty'] += 1
        else:
            self.BOM["parts"][part_id]={'name': part, 'color': color, 'qty': 1}
        # print(f'{self.BOM}')

    def printBOM(self, isSubModel = 0):
        tab=' '*(2*isSubModel)
        if isSubModel == 0:
            print(f'MODEL : {self.name}')
            print('--------------------')
        else:
            print(f'{tab}{self.name}')
            print(f'{tab}----------')
        for id, p in self.BOM["parts"].items():
            print (f'{tab}{p["name"]}({parts[id].colorname}): {p["qty"]} ')
        for sm in self.BOM["submodels"]:
            submodels[sm].printBOM(isSubModel+1)

class LegoPart:
    def __init__(self, partname: str, color: int):
        self.partname = partname.split('.')[0]
        self.color = color
        c = getColorInfo(color)
        self.blcolor = c['BL Color Code'] 
        self.colorname = c['BL Color Name']
    
    def getPartInfo(self):
        p = getPartInfo(self.partname, self.blcolor)
        self.description = p['description']
        self.weight = p['weight']
        self.dimension = p['dimension']
        self.image = p['image']
        self.url = p['url']
        self.unknow = True if (self.description == '') else False



In [11]:

l = model_file.readline().strip().decode('utf-8-sig').split(' ')

model = Model(' '.join(l[2:]))
# throw away next 4 lines
model_file.readline()
model_file.readline()
model_file.readline()

print (f'Creating BOM for model : {model.name}')

thisModel = model

for l in model_file.readlines():

    line = l.strip().decode('utf-8-sig').split(' ')
    # check for line type 
    if line[0] == '0':
        if line[1] == 'FILE':
            sm_name = " ".join(line[2:])
            sm_id = sm_name.lower()
            if sm_id not in submodels:
                submodels[sm_id] = Model(sm_name)
            # if model is created from a line "1" the name is lowercase
            # rewrite name with the one defined on FILE 
            submodels[sm_id].name = sm_name # rewrite name 
            thisModel = submodels[sm_id]
            #print (f'Create submodel : {thisModel.name}')
    elif line[0] == '1':
        color = int(line[1]) # color
        part = " ".join(line[14:]) # part
        if color == 16:    # this is a submodel in the BOM
            thisModel.addSubmodel(part)
        elif color == 24:
            pass
        else:
            thisModel.addPart(part.split('.')[0], color) # part name can be a *.dat so keep only the part name

#for p in parts.values():
#    print(f'{p.partname} - Color: {p.color} ({p.colorname})')
    
    

Creating BOM for model : 10281-bags
10281-bags.addSubmodel(bag 1)
10281-bags.addSubmodel(bag 2)
10281-bags.addSubmodel(bag 3)
10281-bags.addSubmodel(bag 4)
10281-bags.addSubmodel(bag 5)
10281-bags.addSubmodel(bag 6)
10281-bags.addSubmodel(unbagged)
Bag 1.addSubmodel(transparent baglet 1a)
Bag 2.addSubmodel(transperent baglet 2a)
Bag 3.addSubmodel(transparent baglet 3a)
Bag 4.addSubmodel(transparent baglet 4a)
Bag 5.addSubmodel(transparent baglet 5a)
Bag 5.addSubmodel(transparent baglet 5b)
Bag 6.addSubmodel(transparent baglet 6a)


In [12]:
model.printBOM()


MODEL : 10281-bags
--------------------
  Bag 1
  ----------
  3036(Dark Tan): 2 
  2445(Black): 4 
  4477(Dark Bluish Gray): 4 
  87609(Dark Bluish Gray): 6 
  61485(White): 1 
  30414(Light Bluish Gray): 10 
  87079(Black): 12 
  15068(Black): 24 
  3623(Light Bluish Gray): 4 
  59895(Black): 5 
  96874(Dark Turquoise): 1 
    Transparent baglet 1a
    ----------
    3022(Black): 4 
    40145(Black): 4 
    60478(Black): 4 
    99206(Light Bluish Gray): 2 
    3005(Light Bluish Gray): 4 
    27263(White): 4 
    6141(White): 9 
  Bag 2
  ----------
  11213(Dark Tan): 1 
  3795(Dark Brown): 2 
  3460(Dark Brown): 1 
  3666(Reddish Brown): 1 
  30099(Reddish Brown): 2 
  65473(Reddish Brown): 4 
  3004(Reddish Brown): 2 
  88292(Reddish Brown): 2 
  60477(Dark Brown): 4 
  93606(Dark Brown): 1 
  3023(Dark Brown): 4 
  15068(Reddish Brown): 3 
  2420(Reddish Brown): 2 
  3021(Black): 2 
  99206(Reddish Brown): 1 
  30166(Reddish Brown): 2 
  35044(Reddish Brown): 2 
  44728(Reddish Bro

In [13]:
print('Getting parts info from Bricklink')
for p in parts.values():
    p.getPartInfo()
    if p.unknow: 
        print(f"{p.partname} {p.colorname}({p.blcolor})",end='')
        print(" !! UNKNOWN !!")
    elif False:
        print(f"{p.partname} {p.colorname}({p.blcolor})",end='')
        print(" DONE")

Getting parts info from Bricklink


InvalidURL: URL can't contain control characters. '/v2/catalog/catalogitem.page?P=96874 |' (found at least ' ')

# Markdown File Creation

In [ ]:
model.BOM


In [ ]:
table_header = "Picture | Qty | Code | Description | Color \n--------|----:|------|-------------|-------\n"

def table_row(n :str ,p: dict) -> str:
    item = parts[n].partname
    qty = p['qty']
    description = parts[n].description
    color = parts[n].colorname
    # image = f"![{item}]({parts[n].image})"
    image = f'<img src="{parts[n].image}" width="100px">'
    link = f"[{item}]({parts[n].url})"
    return f"{image}| {qty} | {link} | {description} | {color}\n"

def bom2md(mdlname: str, mdlbom : dict, level = 0):
    out = ''
    out += f"{'#'*level}# {mdlname.upper()}\n\n"
    # check if there are parts in the main model
    if mdlbom["parts"]:
        out += table_header
        for n, p in mdlbom["parts"].items():
            out += table_row(n, p)
        out += "\n"
    # create Bom for each submodel
    for smn, smbom in mdlbom["submodels"].items():
        out += bom2md(smn, smbom, level+1)
    return out

md = bom2md(model.name, model.BOM)

display(Markdown(md))

In [ ]:
outmd = "## Here is your markdown block\n"
outmd += "Copy next code block and paste in your _md_ file\n"
outmd += "```(copy)\n"
outmd += md
outmd += "```\n"

display(Markdown(outmd))